# The First Plot

In [2]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
x = [1, 2, 3, 4]
y = [2, 4, 6, 8]

p = figure()

p.circle(x, y, size=10, color='red', legend='circle')
p.line(x, y, color='blue', legend='line')
p.triangle(y, x, color='gold', size=10, legend='triangle')

p.legend.click_policy='hide'
# could also use 'mute' instead of 'hide'

show(p)

# The WWII THOR Data

In [4]:
import pandas as pd

In [6]:
df = pd.read_csv('thor_wwii.csv')
df.head()

,MSNDATE,THEATER,COUNTRY_FLYING_MISSION,NAF,UNIT_ID,AIRCRAFT_NAME,AC_ATTACKING,TAKEOFF_BASE,TAKEOFF_COUNTRY,TAKEOFF_LATITUDE,TAKEOFF_LONGITUDE,TGT_COUNTRY,TGT_LOCATION,TGT_LATITUDE,TGT_LONGITUDE,TONS_HE,TONS_IC,TONS_FRAG,TOTAL_TONS
0,03/30/1941,ETO,GREAT BRITAIN,RAF,84 SQDN,BLENHEIM,10.0,NaN,NaN,NaN,NaN,ALBANIA,ELBASAN,41.10,20.07,0.0,0.0,0.0,0.0
1,11/24/1940,ETO,GREAT BRITAIN,RAF,211 SQDN,BLENHEIM,9.0,NaN,NaN,NaN,NaN,ALBANIA,DURAZZO,41.32,19.45,0.0,0.0,0.0,0.0
2,12/04/1940,ETO,GREAT BRITAIN,RAF,211 SQDN,BLENHEIM,9.0,NaN,NaN,NaN,NaN,ALBANIA,TEPELENE,40.30,20.02,0.0,0.0,0.0,0.0
3,12/31/1940,ETO,GREAT BRITAIN,RAF,211 SQDN,BLENHEIM,9.0,NaN,NaN,NaN,NaN,ALBANIA,VALONA,40.47,19.49,0.0,0.0,0.0,0.0
4,01/06/1941,ETO,GREAT BRITAIN,RAF,211 SQDN,BLENHEIM,9.0,NaN,NaN,NaN,NaN,ALBANIA,VALONA,40.47,19.49,0.0,0.0,0.0,0.0


In [7]:
df.columns.tolist()

['MSNDATE',
 'THEATER',
 'COUNTRY_FLYING_MISSION',
 'NAF',
 'UNIT_ID',
 'AIRCRAFT_NAME',
 'AC_ATTACKING',
 'TAKEOFF_BASE',
 'TAKEOFF_COUNTRY',
 'TAKEOFF_LATITUDE',
 'TAKEOFF_LONGITUDE',
 'TGT_COUNTRY',
 'TGT_LOCATION',
 'TGT_LATITUDE',
 'TGT_LONGITUDE',
 'TONS_HE',
 'TONS_IC',
 'TONS_FRAG',
 'TOTAL_TONS']

## Bokeh ColumnDataSource

In [8]:
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool

In [9]:
sample = df.sample(500)
source = ColumnDataSource(sample)

p = figure()
p.circle(x='AC_ATTACKING', y='TOTAL_TONS',  source=source, size=3, color='blue')
p.title.text = 'Attacking Aircraft and Munitions Dropped'
p.xaxis.axis_label = 'Number of Attacking Aircraft'
p.yaxis.axis_label = 'Tons of Munitions Dropper'

hover = HoverTool()
hover.tooltips = [
    ('Attack Date', '@MSNDATE'),
    ('Attacking Aircraft', '@AC_ATTACKING'),
    ('Tons of Munitions', '@TOTAL_TONS'),
    ('Type of Aircraft', '@AIRCRAFT_NAME')
]
p.add_tools(hover)

show(p)

## Munitions Dropped by Country

In [10]:
from bokeh.palettes import Spectral5
from bokeh.transform import factor_cmap

In [11]:
grouped = df.groupby('COUNTRY_FLYING_MISSION')['TOTAL_TONS', 'TONS_HE', 
                                               'TONS_IC', 'TONS_FRAG'].sum()
print(grouped)

                        TOTAL_TONS     TONS_HE     TONS_IC  TONS_FRAG
COUNTRY_FLYING_MISSION                                               
AUSTRALIA                   479.89      453.90      13.600      18.64
GREAT BRITAIN           1112598.95   868277.23  209036.158    1208.00
NEW ZEALAND                2629.06     4263.70     166.500       0.00
SOUTH AFRICA                 11.69       15.00       0.000       0.00
USA                     1625487.68  1297955.65  205288.200  127655.98


In [12]:
grouped = grouped/1000

In [13]:
source = ColumnDataSource(grouped)
countries = source.data['COUNTRY_FLYING_MISSION'].tolist()
p = figure(x_range=countries)

color_map = factor_cmap(field_name='COUNTRY_FLYING_MISSION', 
                    palette=Spectral5, factors=countries)

p.vbar(x='COUNTRY_FLYING_MISSION', top='TOTAL_TONS', source=source, 
       width=0.70, color=color_map)

p.title.text ='Munitions Dropped by Allied Country'
p.xaxis.axis_label = 'Country'
p.yaxis.axis_label = 'Kilotons of Munitions'

hover = HoverTool()
hover.tooltips = [
    ("Totals", "@TONS_HE High Explosive / @TONS_IC Incendiary / @TONS_FRAG Fragmentation")]

hover.mode = 'vline'
p.add_tools(hover)

show(p)

### Redo the graph
This is a terrible figure, we should do a log10 transform to get a better idea of how this works.

In [14]:
import numpy as np

In [15]:
grouped = df.groupby('COUNTRY_FLYING_MISSION')['TOTAL_TONS', 'TONS_HE', 
                                               'TONS_IC', 'TONS_FRAG'].sum()
log_group = np.log(1+grouped)
print(log_group)

                        TOTAL_TONS    TONS_HE    TONS_IC  TONS_FRAG
COUNTRY_FLYING_MISSION                                             
AUSTRALIA                 6.175639   6.120078   2.681022   2.977568
GREAT BRITAIN            13.922210  13.674267  12.250267   7.097549
NEW ZEALAND               7.874762   8.358127   5.120983   0.000000
SOUTH AFRICA              2.540814   2.772589   0.000000   0.000000
USA                      14.301319  14.076302  12.232175  11.757102


In [16]:
source = ColumnDataSource(log_group)
#countries = source.data['COUNTRY_FLYING_MISSION'].tolist()
p = figure(x_range=countries)

#color_map = factor_cmap(field_name='COUNTRY_FLYING_MISSION', 
#                    palette=Spectral5, factors=countries)

p.vbar(x='COUNTRY_FLYING_MISSION', top='TOTAL_TONS', source=source, 
       width=0.70, color=color_map)

p.title.text ='Munitions Dropped by Allied Country'
p.xaxis.axis_label = 'Country'
p.yaxis.axis_label = 'log(Tons of Munitions)'

hover = HoverTool()
hover.tooltips = [
    ("Totals", "@TONS_HE High Explosive / @TONS_IC Incendiary / @TONS_FRAG Fragmentation")]

hover.mode = 'vline'
p.add_tools(hover)

show(p)

## Types of Munitions Dropped by Country

In [17]:
thor = df.copy()
filter = df['COUNTRY_FLYING_MISSION'].isin(('USA', 'GREAT BRITAIN'))
df = df[filter]

In [18]:
grouped = df.groupby('COUNTRY_FLYING_MISSION')['TOTAL_TONS', 'TONS_HE', 
                                               'TONS_IC', 'TONS_FRAG'].sum()
grouped = grouped / 1000

source = ColumnDataSource(grouped)
countries = source.data['COUNTRY_FLYING_MISSION'].tolist()
p = figure(x_range=countries)

In [20]:
from bokeh.palettes import Spectral3

In [19]:
p.vbar_stack(stackers=['TONS_HE', 'TONS_FRAG', 'TONS_IC'], 
             x='COUNTRY_FLYING_MISSION', source=source, 
             legend = ['High Explosive', 'Fragmentation', 'Incendiary'],
             width=0.5, color=Spectral3)
p.title.text ='Types of Munitions Dropped by Allied Country'
p.legend.location = 'top_left'

p.xaxis.axis_label = 'Country'
p.xgrid.grid_line_color = None	#remove the x grid lines

p.yaxis.axis_label = 'Kilotons of Munitions'

show(p)

## Time-Series and Annotations: Bombing Operations over Time

In [21]:
df = thor

#make sure MSNDATE is a datetime format
df['MSNDATE'] = pd.to_datetime(df['MSNDATE'], format='%m/%d/%Y')

grouped = df.groupby('MSNDATE')['TOTAL_TONS', 'TONS_IC', 'TONS_FRAG'].sum()
grouped = grouped/1000

source = ColumnDataSource(grouped)

p = figure(x_axis_type='datetime')

p.line(x='MSNDATE', y='TOTAL_TONS', line_width=2, source=source, 
       legend='All Munitions')
p.line(x='MSNDATE', y='TONS_FRAG', line_width=2, source=source, color=Spectral3[1], 
       legend='Fragmentation')
p.line(x='MSNDATE', y='TONS_IC', line_width=2, source=source, color=Spectral3[2], 
       legend='Incendiary')

p.yaxis.axis_label = 'Kilotons of Munitions Dropped'

show(p)

## Resampling Time-Series Data

We can use the `groupby` function and accumulate by year, month, week, day, hour, etc. 

In [22]:
grouped = df.groupby(pd.Grouper(key='MSNDATE', freq='M'))['TOTAL_TONS','TONS_IC', 
                                                          'TONS_FRAG'].sum()

In [23]:
grouped = grouped/1000

source = ColumnDataSource(grouped)

p = figure(x_axis_type='datetime')

p.line(x='MSNDATE', y='TOTAL_TONS', line_width=2, source=source, 
       legend='All Munitions')
p.line(x='MSNDATE', y='TONS_FRAG', line_width=2, source=source, color=Spectral3[1], 
       legend='Fragmentation')
p.line(x='MSNDATE', y='TONS_IC', line_width=2, source=source, color=Spectral3[2], 
       legend='Incendiary')

p.yaxis.axis_label = 'Kilotons of Munitions Dropped'

show(p)

### Subsampling
The tutorial suggest subsampling since the daily data are so noisy. That might create trends, of course, but it throws away information. Why not take the mean or sum over week, month, or quarter instead?

In [ ]:
grouped

In [ ]:
monthyear = grouped.groupby([lambda x: x.year, lambda x: x.month]).sum()
monthyear.head()

In [ ]:
monthyear.columns.tolist()

In [ ]:
my = grouped.groupby([grouped.index.year, grouped.index.month]).sum()
my.head()

In [ ]:
grouped['Year'] = grouped.index.year
grouped['Month'] = grouped.index.month
grouped['Day'] =  grouped.index.day

In [ ]:
ymd = grouped[['Year', 'Month']].copy()
ymd['Day'] = 1
ymd.head()

In [ ]:
#make sure MSNDATE is a datetime format
df['MSNDATE'] = pd.to_datetime(df['MSNDATE'], format='%m/%d/%Y')

In [ ]:
import time
import datetime
ymd['Date'] = datetime.date(year=ymd.Year, month=ymd.Month, day=1)

In [ ]:
datetime.date(year=1939, month=9, day=1)

In [ ]:
source = ColumnDataSource(monthyear)

p = figure(x_axis_type='datetime')

p.line(x='MSNDATE', y='TOTAL_TONS', line_width=2, source=source, 
       legend='All Munitions')
p.line(x='MSNDATE', y='TONS_FRAG', line_width=2, source=source, color=Spectral3[1], 
       legend='Fragmentation')
p.line(x='MSNDATE', y='TONS_IC', line_width=2, source=source, color=Spectral3[2], 
       legend='Incendiary')

p.yaxis.axis_label = 'Kilotons of Munitions Dropped'
